In [ ]:
import os
import pickle

import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pygmo as pg
# import seaborn as sns
import tellurium as te

os.chdir('..')

from mpl_toolkits.axes_grid1 import make_axes_locatable
from src.odbm.odbm_main import ModelBuilder

In [ ]:
r_VN = te.loads('src/frenda_brenda/Files/SBML models/240625_VN_biosyn.sbml')

In [ ]:
r_VN.setValue('init(dilution_factor)',1/200)
addedmetabs = pd.read_csv('src/frenda_brenda/adding_biosynthesis.csv')
subs_df = addedmetabs[addedmetabs['Type']=='Metabolite'].drop(columns=['Type','EC','Species'])
for sub, conc in zip(subs_df.KEGG, subs_df.Concentration):
    r_VN.setValue(f'[{sub}]',conc)

Iterate through all factor labels for enzymes and metabolites. Condense into one variable called parameter_labels

In [ ]:
pvals = {p:r_VN[p] for p in r_VN.getGlobalParameterIds() if 'p_EC' in p}
plabels = list(pvals.keys())

metabolites = [s for s in r_VN.getFloatingSpeciesIds() if 'EC' not in s]
rstr = r_VN.getSBML()
vlabels = [v for v in [(rstr[rstr.find('<initialAssignment symbol="'+s+'">')+150:rstr.find('<initialAssignment symbol="'+s+'">')+150+len(s)]).split(' ')[0] for s in metabolites] if 'v' in v]

parameter_labels = plabels + vlabels

Make a dataframe of all possible perturbations of starting conditions of enzymes and metabolites in the system. For actual use will need to change the IDs variable and the perturbation_factors variable.

In [ ]:
# Example list of IDs and perturbation factors
IDs = parameter_labels[:2]
perturbation_factors = [1/10, 10]

# Initialize empty dataframe to store results
concentration_df = pd.DataFrame(columns=['Perturbation ID'] + IDs)

# Generate all combinations of perturbation factors for the IDs
perturbation_combinations = list(itertools.product(perturbation_factors, repeat=len(IDs)))

perturbation_id = 1
for combination in perturbation_combinations:
    # Track the concentration values for this perturbation
    concentration_values = [perturbation_id]
    for i, ID in enumerate(IDs):
        initial_concentration = r_VN.getValue(ID)
        perturbed_concentration = initial_concentration * combination[i]
        concentration_values.append(perturbed_concentration)
    concentration_df.loc[len(concentration_df)] = concentration_values
    perturbation_id += 1

How to call SBML_Barebone_Multi_Fly

In [ ]:
SBML_Barebone_Multi_Fly(model:list, parameter_labels, timepoint, variables:list)

Edited the class of SBML_Barebone_Multi_Fly to include a way to collect metrics on each compound within the system. I am still not too sure how this code is supposed to process all of the different models with different starting conditions. I also want to make sure the metrics it returns go somewhere accessible after being run.

In [ ]:
class SBML_Barebone_Multi_Fly:
    class ModelStuff:
        def __init__(self, model, parameter_labels, variables):
            r = te.loadSBMLModel(model)
            self.species_labels = np.array(r.getFullStoichiometryMatrix().rownames)
            self.r_parameter_labels = np.array(r.getGlobalParameterIds())
            self.parameter_order = np.int32(np.squeeze(np.array([np.where(p == self.r_parameter_labels) for p in parameter_labels if p in self.r_parameter_labels])))
            self.parameter_present = [p in self.r_parameter_labels for p in parameter_labels]
            self.variable_order = {sample:np.int32(np.squeeze(np.array([np.where(p == self.r_parameter_labels) for p in var.keys() if p in self.r_parameter_labels]))) for sample,var in variables.items()}
            self.variable_present = {sample:[p in self.r_parameter_labels for p in var.keys()] for sample,var in variables.items()}
            del r

    def __init__(self, model:list, parameter_labels, timepoint, variables:list):
        self.model = model # now a list of models
        self.timepoint = timepoint
        self.parameter_labels = parameter_labels # all parameters across all models, only the ones that are going to be fitted
        self.variables = variables # list of dict of labels and values. use this for species
        self.cvode_timepoints = 1000
        self.model_stuff = [self.ModelStuff(m, self.parameter_labels, var) for m,var in zip(self.model, self.variables)]

    def _setup_rr(self): # run on engine
        from roadrunner import Config, RoadRunner, Logger
        Logger.disableLogging()
        Config.setValue(Config.ROADRUNNER_DISABLE_PYTHON_DYNAMIC_PROPERTIES, True)
        Config.setValue(Config.LOADSBMLOPTIONS_RECOMPILE, False) 
        Config.setValue(Config.LLJIT_OPTIMIZATION_LEVEL, 4)
        Config.setValue(Config.LLVM_SYMBOL_CACHE, True)
        Config.setValue(Config.LOADSBMLOPTIONS_OPTIMIZE_GVN, True)
        Config.setValue(Config.LOADSBMLOPTIONS_OPTIMIZE_CFG_SIMPLIFICATION, True)
        Config.setValue(Config.LOADSBMLOPTIONS_OPTIMIZE_INSTRUCTION_COMBINING, True)
        Config.setValue(Config.LOADSBMLOPTIONS_OPTIMIZE_DEAD_INST_ELIMINATION, True)
        Config.setValue(Config.LOADSBMLOPTIONS_OPTIMIZE_DEAD_CODE_ELIMINATION, True)
        Config.setValue(Config.LOADSBMLOPTIONS_OPTIMIZE_INSTRUCTION_SIMPLIFIER, True)
        Config.setValue(Config.SIMULATEOPTIONS_COPY_RESULT, True)
        self.r = []
        for m in self.model:
            r = te.loadSBMLModel(m)
            r.integrator.absolute_tolerance = 1e-8
            r.integrator.relative_tolerance = 1e-8
            r.integrator.maximum_num_steps = 2000
            self.r.append(r)
            
    def _simulate(self, x):
        from roadrunner import Config, RoadRunner, Logger
        Logger.disableLogging()
        Config.setValue(Config.ROADRUNNER_DISABLE_PYTHON_DYNAMIC_PROPERTIES, True)
        Config.setValue(Config.LOADSBMLOPTIONS_RECOMPILE, False) 
        Config.setValue(Config.LLJIT_OPTIMIZATION_LEVEL, 4)
        Config.setValue(Config.LLVM_SYMBOL_CACHE, True)
        Config.setValue(Config.LOADSBMLOPTIONS_OPTIMIZE_GVN, True)
        Config.setValue(Config.LOADSBMLOPTIONS_OPTIMIZE_CFG_SIMPLIFICATION, True)
        Config.setValue(Config.LOADSBMLOPTIONS_OPTIMIZE_INSTRUCTION_COMBINING, True)
        Config.setValue(Config.LOADSBMLOPTIONS_OPTIMIZE_DEAD_INST_ELIMINATION, True)
        Config.setValue(Config.LOADSBMLOPTIONS_OPTIMIZE_DEAD_CODE_ELIMINATION, True)
        Config.setValue(Config.LOADSBMLOPTIONS_OPTIMIZE_INSTRUCTION_SIMPLIFIER, True)
        Config.setValue(Config.SIMULATEOPTIONS_COPY_RESULT, True)

        all_results = []
        for r,ms,v in zip(self.r, self.model_stuff, self.variables):

            # this sets the "parameters"
            r.model.setGlobalParameterValues([*ms.parameter_order, *ms.variable_order], [*x[ms.parameter_present], *np.array(list(v.values()))[ms.variable_present]])
            r.reset()

            # this sets species inital concentrations
            for label, value in v.items():
                if not np.isnan(value):
                    if label in ms.species_labels:
                        r.setValue('['+label+']', value)
            try:
                results = r.simulate(0,self.timepoint,self.cvode_timepoints)
            except Exception as e:
                print(e)
                # break # stop if any fail
            r.resetToOrigin()
            all_results.append(results)
        del Config, RoadRunner, Logger, results
        return all_results

    def _calculate_metrics(self, x): # x is an array of parameter values, variables is a list of dictionaries
        all_results =  self._simulate(x) # this returns a list of results

        all_metrics = []
        for result in all_results:
            # result is a numpy array 
            # make a dataframe of the simulation results
            df_un = pd.DataFrame(result, columns=result.colnames)
            columns_to_keep = ['time'] + [col for col in df_un.columns if col.startswith('[C')]
            df = df_un[columns_to_keep]
            
            # create a list to store each row as a dictionary
            rows = []
            
            for compound in df.columns:
                if compound == 'time':
                    continue
                # calculate the initial concentration
                initialconc = df[compound].iloc[0]
                # calculate the final concentration
                finalconc = df[compound].iloc[-1]
                # calculates change in malate from start to finish
                deltatot = finalconc - initialconc
                # finds the minimum concentration and time at minima
                minconc = min(df[compound])
                mintime = df['time'][df[compound].idxmin()]
                # finds the maximum concentration and time at maximum
                maxconc = max(df[compound])
                maxtime = df['time'][df[compound].idxmax()]
                # calculates change in malate from start to max
                deltamax = maxconc - initialconc
                # calculates half of produced malate
                halfmax = deltamax / 2
                # finds the concentration and time closest to half max
                df_closest = df.iloc[(df[compound] - (initialconc + halfmax)).abs().argsort()[:1]]
                halftime = df_closest['time'].iloc[0]
                halfconc = df_closest[compound].iloc[0]
                # append the calculated metrics to the list of rows
                rows.append({'Species': compound,
                             'Final Concentration': finalconc,
                             'Min Conc': minconc,
                             'Max Conc': maxconc,
                             'Min Time': mintime,
                             'Max Time': maxtime,
                             'Total Production': deltatot,
                             'Production to Max': deltamax,
                             'Half Max Time': halftime,
                             'Half Max Conc': halfconc})

        # create a dataframe from the list of rows
        df_final = pd.DataFrame(rows)
            
        all_metrics.append(df_final)

        return all_metrics

    # gotta keep these around but we dont use them
    def fitness(self, x):
        return [1]

    def get_bounds(self):
        return ([0 for i in self.parameter_labels], [1 for i in self.parameter_labels])
    